In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import uuid
import logging
import getpass
import numpy as np
import cv2
import os
import gc
import time
import imageio
import random
import copy
import glob
import json
import shutil
import pickle
import traceback
from collections import Counter
from pprint import pprint as print

In [3]:
from tqdm import tqdm, tqdm_notebook
import requests

In [4]:
from test_api import CLI, CVAT_API_V1, ResourceType

In [5]:
class Args:
    
    def __init__(self, action="create"):
        
        self.action = action
        self.auth = ("", "")  # type tuple
        self.server_host = ""
        self.server_port = 8080
        self.https = False
        self.loglevel = 20

        self.name = "test"
        self.labels = [{"name":"text", "attributes": []}]  # type list
        self.overlap = 0
        self.segment_size = 0
        self.bug = ""
        self.resource_type = ResourceType(0)
        self.resources = ["/home/phan.huy.hoang/Pictures/test_img_cvat.zip"]
        self.annotation_path = "/home/phan.huy.hoang/cvat/utils/cli/instances_default.json"
        self.annotation_format = "COCO 1.0"

        # for git only
        self.completion_verification_period = 10
        self.dataset_repository_url = ""
        self.lfs = False

args = Args()

In [6]:
CVAT_URL = "http://{}:{}/".format(args.server_host, args.server_port)
API_URL = CVAT_URL + "api/v1/"
username, password = args.auth
username, password

('hoangph', 'hoangph_Aa@123456')

In [26]:
def get_cookies():
    login_json = {"username": username, "password": password}
    login_req = requests.post(API_URL + 'auth/login', json=login_json)
    cookies = login_req.cookies
    return cookies

cookies = get_cookies()

In [ ]:
req = requests.get("{}/api/v1/tasks/530".format(CVAT_URL), cookies=cookies)
req.text

### List all available tasks

- stamp + table detection: id from 325 -> 530

In [9]:
with requests.Session() as session:
    api = CVAT_API_V1('{}:{}'.format(args.server_host, args.server_port), args.https)
    cli = CLI(session, api, args.auth)

In [10]:
res = cli.tasks_list(use_json_output=True)

In [11]:
len(res)  # number of tasks

206

In [12]:
task_ids = [i["id"] for i in res]

In [13]:
task_ids[0]

530

### Download annotated data from each task (only annotation files)

In [14]:
task_id = task_ids[0]

In [15]:
# change save folder <<<<<
saved_dir = "./dump/annotated_data/stamp_table"

In [16]:
# download as zip file
res = cli.tasks_dump(task_id, "COCO 1.0", os.path.join(saved_dir, "{}.zip".format(task_id)))

In [18]:
# file will save in the folder: $saved_dir/530.zip

In [19]:
ls $saved_dir

530.zip  annotations/  test.zip


In [ ]:
for task_id in tqdm_notebook(task_ids):
    cli.tasks_dump(task_id, "COCO 1.0", os.path.join(saved_dir, "{}.zip".format(task_id)))

### Download both image + annotation

In [55]:
saved_dir = "./dump/annotated_data/stamp_table"

In [56]:
def download_file(req, task_id):
    with open(os.path.join(saved_dir, '{}.zip'.format(task_id)), 'wb') as f:
        for chunk in req.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    print("{} download completed".format(task_id))

In [ ]:
# download all data (image + annotation)
for task_id in task_ids:
    url = url_fmt.format(task_id)
    
    while True:
        req = requests.get(url, cookies=cookies)
        if req.status_code == 200:
            download_file(req, task_id)
            break